# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filename = "output_data/cities.csv"
df = pd.read_csv(filename)
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,42.91,61,75,6.91,CL,1643948998
1,1,caronport,50.4501,-105.8179,-12.32,99,93,7.14,CA,1643948998
2,2,ushuaia,-54.8000,-68.3000,40.66,75,20,13.80,AR,1643948999
3,3,olafsvik,64.8945,-23.7142,27.55,74,93,25.86,IS,1643948999
4,4,hithadhoo,-0.6000,73.0833,81.73,74,100,10.11,MV,1643948999
...,...,...,...,...,...,...,...,...,...,...
520,520,poso,-1.3959,120.7524,86.99,58,92,4.94,ID,1643949805
521,521,houma,29.5958,-90.7195,46.00,97,100,13.22,US,1643949806
522,522,santiago del estero,-27.7951,-64.2615,76.15,77,55,8.59,AR,1643949806
523,523,abadan,30.3392,48.3043,58.95,88,75,11.50,IR,1643949806


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = df[['Lat','Lng']]
weights = df['Humidity'].astype(float)
intensity = df['Humidity'].max()
fig = gmaps.figure(center = [0,0], zoom_level = 2)

heatmap = gmaps.heatmap_layer(locations, weights, max_intensity = intensity, point_radius = 1.5, dissipating = False)

fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
ideal_weather = df[(df["Wind Speed"] >= 5) & (df["Wind Speed"] <= 15) & (df["Cloudiness"] >= 30) & (df["Cloudiness"] <= 45) & 
   (df["Max Temp"] >= 65) & (df["Max Temp"] <= 80)] 
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,bathsheba,13.2167,-59.5167,75.81,73,40,11.50,BB,1643948878
77,77,cabo san lucas,22.8909,-109.9124,67.51,57,34,5.64,MX,1643948871
281,281,lakes entrance,-37.8811,147.9810,70.16,59,32,9.60,AU,1643949441
326,326,okato,-39.2000,173.8833,73.69,72,39,12.12,NZ,1643949513
340,340,kingaroy,-26.5333,151.8333,78.84,53,40,11.50,AU,1643949517
392,392,oistins,13.0667,-59.5333,75.29,73,40,11.50,BB,1643949389


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df = ideal_weather.reset_index(drop = True)
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,76,bathsheba,13.2167,-59.5167,75.81,73,40,11.50,BB,1643948878,
1,77,cabo san lucas,22.8909,-109.9124,67.51,57,34,5.64,MX,1643948871,
2,281,lakes entrance,-37.8811,147.9810,70.16,59,32,9.60,AU,1643949441,
3,326,okato,-39.2000,173.8833,73.69,72,39,12.12,NZ,1643949513,
4,340,kingaroy,-26.5333,151.8333,78.84,53,40,11.50,AU,1643949517,
5,392,oistins,13.0667,-59.5333,75.29,73,40,11.50,BB,1643949389,


In [11]:
parameters = {"radius": 5000,
              "types": "hotel",
              "keyword": "hotels",
              "key": g_key
    
}

In [38]:
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    parameters["location"] = f"{latitude},{longitude}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(url,params = parameters)
    hotelname = hotels.json()
    hotel_df.loc[index, 'Hotel Name'] = hotelname["results"][0]["name"]
    
#    print(hotelname)

In [46]:
#Cleaned
hotel_df = hotel_df.drop('Unnamed: 0',axis = 1)

In [47]:
hotel_df
hotelfilename = "output_data/hotels.csv"
hotel_df.to_csv(hotelfilename,index = False) 

In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_info

In [51]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker)
# Display figure
fig
plt.savefig('hotels.png',dpi = 300)

<Figure size 432x288 with 0 Axes>

In [52]:
from ipywidgets.embed import embed_minimal_html

embed_minimal_html('export.html', views=[fig])
